# Knowledge Intensive NLP Summer School

## Notebook 3

The goals of this notebook are:

* Train a T5 notebook for SQuAD task
* Evaluate what happens when context isn't used
* Explore the Fusion in Decoder model 


## Resources
You can find help for the HuggingFace library from their website: 

* T5 https://huggingface.co/docs/transformers/model_doc/t5
* Datasets https://huggingface.co/docs/datasets/index

## Tutorial

This notebook is based on the following tutorials:

* Fine-tuning https://huggingface.co/docs/transformers/training
* Language Generation https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html


# Prelude

The following code will use the SQuAD dataset and train a model to predict an answer given a question and passage. Have a look at it and familiarize yourself

In [1]:
!pip install xfact

In [2]:
import logging
import os
import sys
import transformers
import datasets 

from transformers import (
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed, AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification
)
from transformers import Seq2SeqTrainer
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from collections import defaultdict
from operator import itemgetter

from xfact.config.args import ModelArguments, DataTrainingArguments
from xfact.logs.comet_callback import CometTrainingCallback
from xfact.logs.logs import setup_logging
from xfact.nlp.dataset import XFactDataset, XFactSeq2SeqDataset
from xfact.nlp.model import ModelFactory
from xfact.nlp.post_processing import PostProcessor
from xfact.nlp.reader import Reader
from xfact.nlp.scoring import Scorer
from xfact.registry.module import import_submodules


check_min_version("4.16.0")
logger = logging.getLogger(__name__)
set_seed(1337)

setup_logging("INFO")
transformers.utils.logging.set_verbosity("INFO")
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()


comet_ml is installed but `COMET_API_KEY` is not set.


In [3]:
model_args = ModelArguments(
    model_name_or_path="t5-small",
)
data_args = DataTrainingArguments(dataset="squad",train_file="train", validation_file="validation")


#NOTE SET MPS to False if error in training
training_args = Seq2SeqTrainingArguments(
    learning_rate=5e-5,
    logging_steps=10,
    logging_strategy="steps",
    eval_steps=10,
    evaluation_strategy="steps",
    do_eval=True,
    do_train=True,
    output_dir="test",
    use_mps_device=True) 

[INFO|training_args.py:1267] 2023-07-05 13:50:40,377 >> Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
[INFO|training_args.py:1669] 2023-07-05 13:50:40,377 >> PyTorch: setting up devices
[INFO|training_args.py:1407] 2023-07-05 13:50:40,436 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [4]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=True,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
)

[INFO|tokenization_utils_base.py:1823] 2023-07-05 13:50:40,920 >> loading file spiece.model from cache at /Users/user/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/spiece.model
[INFO|tokenization_utils_base.py:1823] 2023-07-05 13:50:40,921 >> loading file tokenizer.json from cache at /Users/user/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer.json
[INFO|tokenization_utils_base.py:1823] 2023-07-05 13:50:40,922 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1823] 2023-07-05 13:50:40,923 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:1823] 2023-07-05 13:50:40,924 >> loading file tokenizer_config.json from cache at /Users/user/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer_config.json
[INFO|configuration_utils.py:669] 2023-07-05 13:50:41,425 >> loading confi

In [5]:
# Load the dataset, we are only using 1000 training examples and 100 test for now
squad_dataset = datasets.load_dataset(data_args.dataset, split=["train[:1000]","validation[:100]"])
squad_dataset = {
    "train":squad_dataset[0],
    "validation":squad_dataset[1],
}

2023-07-05 13:50:44,686 [ datasets.builder ][ WARNING ] Found cached dataset squad (/Users/user/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453) (builder.py:835)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
squad_dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [7]:
max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

data_files = {
    "train": data_args.train_file,
    "validation": data_args.validation_file
}

class SQUADWithQuestionAndContext(XFactSeq2SeqDataset):
    def prepare_src(self, instance):
        return instance["question"] + " - " + instance["context"]

    def prepare_tgt(self, instance):
        return instance["answers"]['text'][0] or "No Answer"
        
loaded_datasets = {
    split: SQuADPageTitlePredictionDataset(tokenizer,
                                           squad_dataset[split],
                                           max_seq_length,
                                           name=split,
                                           max_target_length=data_args.max_target_length)
    for split, path in data_files.items()
}


2023-07-05 13:50:44,783 [ xfact.nlp.dataset ][ INFO ] Tokenizer doesn't have a sep token. Create it (dataset.py:51)
2023-07-05 13:50:44,794 [ xfact.nlp.dataset ][ INFO ] Sep token id is 32100 (dataset.py:57)
train: 1000it [00:00, 21170.31it/s]
Generating instances: 106it [00:00, 1056.60it/s]

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? - Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
tensor([  304,  4068,   410,     8, 16823,  3790,     3, 18280,  2385,    16,
          507,  3449,    16,   301,  1211,  1395,  1410,    58,     3,    18,
        30797,   120,     6,     8,   496,    65,     3,     9,  6502,  18

Generating instances: 1000it [00:00, 1203.35it/s]
2023-07-05 13:50:45,675 [ xfact.nlp.dataset ][ INFO ] Output prompt tokens are [] (dataset.py:164)
2023-07-05 13:50:45,695 [ xfact.nlp.dataset ][ INFO ] Sep token id is 32100 (dataset.py:57)
validation: 100it [00:00, 18351.80it/s]
Generating instances: 100it [00:00, 1819.91it/s]
2023-07-05 13:50:45,759 [ xfact.nlp.dataset ][ INFO ] Output prompt tokens are [] (dataset.py:164)


Which NFL team represented the AFC at Super Bowl 50? - Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
tensor([ 4073, 10439,   372,  7283,     8,    71,  5390,    44,  2011,  9713,
          943,    58,     3,    18,  2011,  9713,   943,    47,    46,   797,
         3370

In [10]:
data_collator = lambda batch: XFactSeq2SeqDataset.collate_fn(model, batch, tokenizer.pad_token_id, data_args.ignore_pad_token_for_loss)

def my_metrics_function(eval_predictions):
    print(str(eval_predictions))
    return {"metric_name":0}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=loaded_datasets["train"],
    eval_dataset=loaded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=my_metrics_function,
)


train_result = trainer.train()
trainer.save_model()

metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()


[INFO|trainer.py:1786] 2023-07-05 13:52:05,986 >> ***** Running training *****
[INFO|trainer.py:1787] 2023-07-05 13:52:05,986 >>   Num examples = 1,000
[INFO|trainer.py:1788] 2023-07-05 13:52:05,986 >>   Num Epochs = 3
[INFO|trainer.py:1789] 2023-07-05 13:52:05,987 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:1790] 2023-07-05 13:52:05,987 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:1791] 2023-07-05 13:52:05,987 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1792] 2023-07-05 13:52:05,987 >>   Total optimization steps = 375
[INFO|trainer.py:1793] 2023-07-05 13:52:05,988 >>   Number of trainable parameters = 60,506,624


Step,Training Loss,Validation Loss,Metric Name
10,2.454300,1.047829,0
20,1.993800,0.678388,0
30,1.653100,0.616795,0


[INFO|trainer.py:3200] 2023-07-05 13:52:10,487 >> ***** Running Evaluation *****
[INFO|trainer.py:3202] 2023-07-05 13:52:10,487 >>   Num examples = 100
[INFO|trainer.py:3205] 2023-07-05 13:52:10,487 >>   Batch size = 8


[INFO|trainer.py:3200] 2023-07-05 13:52:17,002 >> ***** Running Evaluation *****
[INFO|trainer.py:3202] 2023-07-05 13:52:17,002 >>   Num examples = 100
[INFO|trainer.py:3205] 2023-07-05 13:52:17,002 >>   Batch size = 8


[INFO|trainer.py:3200] 2023-07-05 13:52:23,458 >> ***** Running Evaluation *****
[INFO|trainer.py:3202] 2023-07-05 13:52:23,458 >>   Num examples = 100
[INFO|trainer.py:3205] 2023-07-05 13:52:23,459 >>   Batch size = 8


KeyboardInterrupt: 

# Exercises

## Seq2seq model T5
1) Adapt the code to report the answer exact match accuracy (see `compute_metrics` function in HuggingFace Trainer)
2) Compare a different version of the task, evaluating whether the model can accurately predict answers without the context paragraph. How does teh exact match answer accuracy change?
3) Train a GENRE-style information retrieval system


## Extension exercises

1) Look at the Forward method in the T5Model class in HuggingFace (https://github.com/huggingface/transformers/blob/ee339bad01bf09266eba665c5f063f0ab7474dad/src/transformers/models/t5/modeling_t5.py#L1414). Explore the fusion in decoder library: https://github.com/facebookresearch/fid - How would you change this method to encode multiple passages separately?
2) Download the FAISS library and adapt code from yesterday's lab to index the DPR-encoded contexts in FAISS and evaluate the speed-up